함수형 API
함수형 API : Permute, Reshape

In [ ]:
# 함수형 API : Permute, Reshape
import tensorflow as tf
import numpy as np

# 2: create 20 input data
A = np.array([[1,2,3,4] , [5,6,7,8]], dtype='float32')
A = A.reshape(1, 2, 4, 1)
print(A)

#3: build a model
x = tf.keras.layers.Input(shape=A.shape[1:]) 
y = tf.keras.layers.Reshape([4, 2, 1])(x)
z = tf.keras.layers.Permute([2, 1, 3])(x)

model = tf.keras.Model(inputs=x, outputs= [y,z])
model.summary()

#4: apply A to model
## output = model(A)    # Tensor output
output = model.predict(A)
print("A[0,:,:,0]=", A[0,:,:,0])
print("output[0]=", output[0][0,:,:,0])
print("output[1]", output[1][0,:,:,0])

[[[[1.]
   [2.]
   [3.]
   [4.]]

  [[5.]
   [6.]
   [7.]
   [8.]]]]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 4, 1)]    0           []                               
                                                                                                  
 reshape (Reshape)              (None, 4, 2, 1)      0           ['input_1[0][0]']                
                                                                                                  
 permute (Permute)              (None, 4, 2, 1)      0           ['input_1[0][0]']                
                                                                                                  
Total params: 0
Trainable params: 0
Non-trainable params: 0
________________________________________________________________

In [ ]:
# UpSampling1D, UpSampling2D
import tensorflow as tf
import numpy as np

# 2: create 20 input data
A = np.array([[1,2,3,4] , [5,6,7,8]], dtype='float32')
A = A.reshape(1, 2, 4, 1) #(batch, rows, cols, channels)

#3: build a model
x = tf.keras.layers.Input(shape=A.shape[1:]) 
y = tf.keras.layers.UpSampling2D()(x)

u = tf.keras.layers.Reshape([8,1])(x)
z = tf.keras.layers.UpSampling1D()(u)

model = tf.keras.Model(inputs=x, outputs= [y,z])
model.summary()

#4: apply A to model
## output = model(A)    # Tensor output
output = model.predict(A)
print("A[0,:,:,0]=", A[0,:,:,0])
print("output[0]=", output[0][0,:,:,0])
print("output[1]", output[1][0,:,0])

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 2, 4, 1)]    0           []                               
                                                                                                  
 reshape_3 (Reshape)            (None, 8, 1)         0           ['input_6[0][0]']                
                                                                                                  
 up_sampling2d_2 (UpSampling2D)  (None, 4, 8, 1)     0           ['input_6[0][0]']                
                                                                                                  
 up_sampling1d_2 (UpSampling1D)  (None, 16, 1)       0           ['reshape_3[0][0]']              
                                                                                            

In [ ]:
# 함수형 API : 1D 병합(merge) 연산

#1:
gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

#2: 1D input data: A, B
A = np.array([1, 2, 3, 4, 5]).astype('float32')
B = np.array([1, 2, 3, 4, 5, 6, 7, 8]).astype('float32')
print(A)
A = np.reshape(A, (1, -1, 1)) #(batch, steps, channels)
B = np.reshape(B, (1, -1, 1)) #(batch, steps, channels)
print(A)

#3: build a model
input_x = tf.keras.layers.Input(shape=A.shape[1:])
input_y = tf.keras.layers.Input(shape=B.shape[1:])

x = tf.keras.layers.MaxPool1D()(input_x)
y = tf.keras.layers.MaxPool1D()(input_y)

pad = y.shape[1] - x.shape[1]
x = tf.keras.layers.ZeroPadding1D(padding=(0, pad))(x)

out2 = tf.keras.layers.Add()([x, y])    # 각 같은 위치의 요소를 더함
out3 = tf.keras.layers.Concatenate()([x, y])  # 열or행에 추가함 
out4 = tf.keras.layers.Dot(axes=[1,1])([x,y])  # Dot은 내적, -1일때 외적 ==> 내적은 pair가 되는 것 끼리 곱해서 더함
out5 = tf.keras.layers.Dot(axes=-1)([x,y])    # 외적은 왼쪽의 한 요소를 오른쪽의 요소와 곱한 뒤 나열

out_list = [x, y, out2, out3, out4, out5]
model = tf.keras.Model(inputs=[input_x, input_y], outputs = out_list)
###model.summary()
print("model.output_shape=", model.output_shape)

#4: apply [A,B] to model
#3output = model([A,B])
output = model.predict([A, B])
for i in range(len(output)):
  print("output[{}]={}".format(i, output[i]))

[1. 2. 3. 4. 5.]
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
model.output_shape= [(None, 4, 1), (None, 4, 1), (None, 4, 1), (None, 4, 2), (None, 1, 1), (None, 4, 4)]
1/1 [==============================] - 0s 46ms/step
output[0]=[[[2.]
  [4.]
  [0.]
  [0.]]]
output[1]=[[[2.]
  [4.]
  [6.]
  [8.]]]
output[2]=[[[4.]
  [8.]
  [6.]
  [8.]]]
output[3]=[[[2. 2.]
  [4. 4.]
  [0. 6.]
  [0. 8.]]]
output[4]=[[[20.]]]
output[5]=[[[ 4.  8. 12. 16.]
  [ 8. 16. 24. 32.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [ ]:
#함수형 API:2D 병합(merge) 연산

#1:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

#2: 2D input data: A, B
A = np.array([[1, 2], [3, 4]]).astype('float32')
A= A.reshape(-1, 2, 2, 1) 
print("A = ", A)
B = np.array([[5, 6], 
             [7, 8]]).astype('float32')
B= B.reshape(-1, 2, 2, 1) 
print(B)

C = np.array([1, 2, 3]).astype('float32')
C = C.reshape(-1, 3, 1, 1)

#3: build a model
x = tf.keras.layers.Input(shape=A.shape[1:])
y = tf.keras.layers.Input(shape=B.shape[1:])
z = tf.keras.layers.Input(shape=C.shape[1:])

out3 = tf.keras.layers.Add()([x, y])
out4 = tf.keras.layers.Concatenate()([x, y])
out5 = tf.keras.layers.Dot(axes = -1)([x, y])
out6 = tf.keras.layers.Dot(axes=-1)([x, y])
out_list = [x, y, z, out3, out4, out5, out6]
model = tf.keras.Model(inputs=[x, y, z], outputs= out_list)
###model.summary()
print("model.output_shape=", model.output_shape)

#4: apply [A,B] to model
#3output = model([A,B])
output = model.predict([A, B, C])
for i in range(len(output)):
  print("output[{}]={}".format(i, output[i]))


A =  [[[[1.]
   [2.]]

  [[3.]
   [4.]]]]
[[[[5.]
   [6.]]

  [[7.]
   [8.]]]]
model.output_shape= [(None, 2, 2, 1), (None, 2, 2, 1), (None, 3, 1, 1), (None, 2, 2, 1), (None, 2, 2, 2), (None, 2, 2, 2, 2), (None, 2, 2, 2, 2)]
1/1 [==============================] - 0s 116ms/step
output[0]=[[[[1.]
   [2.]]

  [[3.]
   [4.]]]]
output[1]=[[[[5.]
   [6.]]

  [[7.]
   [8.]]]]
output[2]=[[[[1.]]

  [[2.]]

  [[3.]]]]
output[3]=[[[[ 6.]
   [ 8.]]

  [[10.]
   [12.]]]]
output[4]=[[[[1. 5.]
   [2. 6.]]

  [[3. 7.]
   [4. 8.]]]]
output[5]=[[[[[ 5.  6.]
    [ 7.  8.]]

   [[10. 12.]
    [14. 16.]]]


  [[[15. 18.]
    [21. 24.]]

   [[20. 24.]
    [28. 32.]]]]]
output[6]=[[[[[ 5.  6.]
    [ 7.  8.]]

   [[10. 12.]
    [14. 16.]]]


  [[[15. 18.]
    [21. 24.]]

   [[20. 24.]
    [28. 32.]]]]]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
import numpy as np
import matplotlib.pyplot as plt

#1:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

#2: 
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1, 1]], dtype = np.float32)

y_and = np.array([[0], [0], [0], [1]], dtype = np.float32)
y_or = np.array([[0], [1], [1], [1]], dtype = np.float32)
y_xor = np.array([[0], [1], [1], [0]], dtype = np.float32)

y_and = tf.keras.utils.to_categorical(y_and)
y_or = tf.keras.utils.to_categorical(y_or)
y_xor = tf.keras.utils.to_categorical(y_xor)

#3: build a model
x_and = Input(shape = (2,))
x =  Dense(units = 2, activation = 'sigmoid')(x_and)
out_or = Dense(units = 2, activation = 'softmax', name = 'and')(x)

x_or = Input(shape = (2, ))
x =  Dense(units = 2, activation = 'sigmoid')(x_or)
out_or = Dense(units = 2, activation = 'softmax', name = 'or')(x)

x_xor = Input(shape = (2, ))
x =  Dense(units = 2, activation = 'sigmoid')(x_or)
out_xor = Dense(units = 2, activation = 'softmax', name = 'xor')(x)

model = tf.keras.Model(inputs = [x_and, x_or, x_xor], outputs = [out_and, out_or, out_xor])
model.summary()

#4: train and evaluate
opt = tf.keras.optimizers.RMSprop(learning_rate = 0.1)
model.compile(optimizer = opt, loss ='mse', metrics = ['accuracy'])
ret = model.fit(x = [X, X, X], y = [y_and, y_or, y_xor], epochs = 100, batch_size = 4, verbose = 0)

test = model.evaluate(x = [X, X, X], y=[y_and, y_or, y_xor], verbose=0)
print('total loss = ', test[0])
print('AND: loss{}, acc={}'.format(test[1]. test[4]))
print('OR: loss={}, acc={}'.format(test[2], test[5]))
print('XOR: loss={}, acc={}'.format(test[3], test[6]))

# graph
plt.plot(ret.history['loss'], 'k--', label='loss')
plt.plot(ret.history['and_loss'], 'r--', label='and_loss')
plt.plot(ret.history['or_loss'], 'g--', label='or_loss')
plt.plot(ret.history['xor_loss'], 'b--', label='xor_loss')

plt.plot(ret.history['and_accuracy'], 'r--', label='and_accuracy')
plt.plot(ret.history['or_accuracy'], 'g--', label='or_accuracy')
plt.plot(ret.history['xor_accuracy'], 'b--', label='xor_accuracy')

plt.xlabel('epochs')
plt.ylabel('loss and accuracy')
plt.legend(loc = 'best')
plt.show()




ValueError: ignored